In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import time
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Ridge, LassoCV,LassoLarsCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from scipy.stats import skew

def RMSLE_(y_val, y_val_pred):
    return np.sqrt(np.mean((np.log(y_val+1)-np.log(y_val_pred+1))**2))
RMSLE = make_scorer(RMSLE_, greater_is_better=False) 

def RMSE_(y_val, y_val_pred):
    return np.sqrt(np.mean((y_val-y_val_pred)**2))
RMSE = make_scorer(RMSE_, greater_is_better=False)

from time import time


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def create_submission(prediction,score,comment):
    now = datetime.datetime.now()
    sub_file = 'submission_'+str(score)+'_'+str(now.strftime("%Y-%m-%d-%H-%M"))+comment+'.csv'
    #sub_file = 'prediction_training.csv'
    print ('Creating submission: ', sub_file)
    pd.DataFrame({'id': id_test, 'price_doc': prediction}).to_csv(sub_file, index=False)

# train need to be test when do test prediction
def data_preprocess(X_train,X_test):
#     outlier_idx = [4,11,13,20,46,66,70,167,178,185,199, 224,261, 309,313,318, 349,412,423,440,454,477,478, 523,540, 581,588,595,654,688, 691, 774, 798, 875, 898,926,970,987,1027,1109, 1169,1182,1239, 1256,1298,1324,1353,1359,1405,1442,1447]
#     train.drop(train.index[outlier_idx],inplace=True)
    all_data = pd.concat((X_train,X_test))
    
    lowerClipCol = ['floor_from_top', 'roomsize', 'extra_area', 'age_at_sale']
    for c in lowerClipCol:
        all_data[[c]]=all_data[[c]].clip(lower=0)
    
# #     to_delete = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
# #     all_data = all_data.drop(to_delete,axis=1)

#     #train["SalePrice"] = np.log1p(train["SalePrice"])
#     #log transform skewed numeric features
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
    skewed_feats = X_train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
    skewed_feats = skewed_feats[skewed_feats > 0.75]
    skewed_feats = skewed_feats.index
    all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
    #all_data = pd.get_dummies(all_data)
    #all_data = all_data.fillna(all_data.median())
    
    all_data.replace([np.infty, -np.infty], np.nan)
    imp = Imputer(missing_values=np.nan, strategy='median')
    all_data=pd.DataFrame(imp.fit_transform(all_data), columns=all_data.columns)
    imp = Imputer(missing_values=np.infty, strategy='median')
    all_data=pd.DataFrame(imp.fit_transform(all_data), columns=all_data.columns)

    X_train = all_data[:X_train.shape[0]]
    X_test = all_data[X_train.shape[0]:]
    
#     imp = Imputer(missing_values=np.nan, strategy='median')
#     imp.fit(all_data)
#     X_train = pd.DataFrame(imp.transform(X_train),columns=X_train.columns)
#     X_test = pd.DataFrame(imp.transform(X_test),columns=X_test.columns)
    
#     imp = Imputer(missing_values=np.infty, strategy='median')
#     imp.fit(pd.concat([X_train,X_test]))
#     X_train = pd.DataFrame(imp.transform(X_train),columns=X_train.columns)
#     X_test = pd.DataFrame(imp.transform(X_test),columns=X_test.columns)

    return X_train,X_test


In [3]:
class ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
    def fit_predict(self,train,test,ytr):
        X = train.values
        y = ytr.values
        T = test.values
        folds = list(KFold(len(y), n_folds = self.n_folds, shuffle = True, random_state = 0))
        S_train = np.zeros((X.shape[0],len(self.base_models)))
        S_test = np.zeros((T.shape[0],len(self.base_models))) 
        for i,reg in enumerate(base_models):
            print ("Fitting the base model...")
            S_test_i = np.zeros((T.shape[0],len(folds))) 
            for j, (train_idx,test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                reg.fit(X_train,y_train)
                y_pred = reg.predict(X_holdout)[:]
                S_train[test_idx,i] = y_pred
                S_test_i[:,j] = reg.predict(T)[:]
            S_test[:,i] = S_test_i.mean(1)
         
        print ("Stacking base models...")
        # tuning the stacker
        param_grid = {
            'alpha': [1e-3,5e-3,1e-2,5e-2,1e-1,0.2,0.3,0.4,0.5,0.8,1e0,3,5,7,1e1],
        }
        grid = GridSearchCV(estimator=self.stacker, param_grid=param_grid, n_jobs=1, cv=5, scoring=RMSE)
        grid.fit(S_train, y)
        try:
            print('Param grid:')
            print(param_grid)
            print('Best Params:')
            print(grid.best_params_)
            print('Best CV Score:')
            print(-grid.best_score_)
            print('Best estimator:')
            print(grid.best_estimator_)
            print(message)
        except:
            pass

        y_pred = grid.predict(S_test)[:]
        return y_pred, -grid.best_score_

In [4]:
train = pd.read_csv('./input/train_further_additional_feature_filtered.csv',index_col=0)
y_train = pd.read_csv('./input/y_train_further_additional_feature_filtered.csv',index_col=0)['price_doc']
test = pd.read_csv('./input/test_further_additional_feature_filtered.csv',index_col=0)
macro = pd.read_csv('./input/macro.csv', parse_dates=['timestamp'])
id_test = pd.read_csv('./input/test.csv').id

# build a model library (can be improved)
base_models = [
        RandomForestRegressor(max_depth=6, n_jobs=1, random_state=0,
            n_estimators=500, max_features=.95
        ), #0.34513
#         RandomForestRegressor(
#             n_jobs=1, random_state=0,
#             n_estimators=500, max_features=20,
#             max_depth = 7
#         ),
#         ExtraTreesRegressor(
#             n_jobs=1, random_state=0, 
#             n_estimators=500, max_features=15
#         ),
#         ExtraTreesRegressor(
#             n_jobs=1, random_state=0, 
#           n_estimators=500, max_features=20
#         ),
        GradientBoostingRegressor(learning_rate=0.02, n_estimators=500, min_samples_leaf=70, 
            min_samples_split=200, max_features='sqrt',max_depth=6,subsample=0.85,random_state=10
            ), #LB 0.32465
#         GradientBoostingRegressor(
#             random_state=0, 
#             n_estimators=500, max_features=15, max_depth=6,
#             learning_rate=0.05, subsample=0.8
#         ),
        XGBRegressor(seed=5, learning_rate=.02, subsample=.95, max_depth=6, min_child_weight=3, 
            n_estimators=500, colsample_bytree=0.95
        ), #0.31625
 
#         XGBRegressor(
#             seed=0,
#             n_estimators=500, max_depth=7,
#             learning_rate=0.05, subsample=0.8, colsample_bytree=0.75
#         ),
        LassoCV(alphas = [1, 0.1, 0.001, 0.0005]),
        KNeighborsRegressor(n_neighbors = 5),
        KNeighborsRegressor(n_neighbors = 10),
        KNeighborsRegressor(n_neighbors = 15),
        KNeighborsRegressor(n_neighbors = 25),
        LassoLarsCV(),
        ElasticNet(),
        SVR()
    ]

ensem = ensemble(
        n_folds=5,
        stacker=Ridge(),
        base_models=base_models
    )

X_train,X_test= data_preprocess(train,test)
y_pred, score = ensem.fit_predict(X_train,X_test,np.log1p(y_train))

create_submission(np.expm1(y_pred),score,"skewAdded")


/home/ubuntu/anaconda2/lib/python2.7/site-packages/scipy/stats/stats.py:926: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - np.expand_dims(np.mean(a, axis), axis)


Fitting the base model...


KeyboardInterrupt: 

In [8]:
if(True):
    X_train = train
    X_test = test
    all_data = pd.concat((X_train,X_test))
    
    lowerClipCol = ['floor_from_top', 'roomsize', 'extra_area', 'age_at_sale']
    for c in lowerClipCol:
        print(sum(all_data[c]<0))
        #all_data[[c]]=all_data[[c]].clip(lower=0)
    
# #     to_delete = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
# #     all_data = all_data.drop(to_delete,axis=1)

#     #train["SalePrice"] = np.log1p(train["SalePrice"])
#     #log transform skewed numeric features
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
    skewed_feats = X_train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
    skewed_feats = skewed_feats[skewed_feats > 5]
    print(len(skewed_feats))

81
9
33
1682
60
